#### Importaciones

In [1]:
# Imports
from dotenv import load_dotenv
load_dotenv()

from ipywidgets import VBox
from IPython.display import display
from ui.components import (
    Toolbar,
    ClusterPlotCreator,
)

from chainsaw.plot import Plot
from chainsaw.model import Tree
from chainsaw.model.node import Unit, Charge
from chainsaw.enum.administration_type import AdministrationType

from chainsaw.db import SessionLocal
from chainsaw.pipeline import (
    Scrapping,
    Cleaning,
    Finding,
    Prompting,
    LLMExtraction,
    Pipeline,
)

#### Carga de datos

In [2]:
# Menú
data = {'tree': {}, 'df': {}}
toolbar, plots, interactive_output = Toolbar(data)
display(toolbar)
display(interactive_output)

In [3]:
# Plots
display(plots, clear=True)

#### Clusters

In [4]:
# Cluster Input
cluster_menu, cluster_plot = ClusterPlotCreator(data)
display(VBox([cluster_menu, cluster_plot]))

#### Distribución de unidades

In [5]:
#year = "2023_12_09"
year = "2025_07_08"

In [10]:
import pandas as pd
from collections import defaultdict

count_by_jurisdiction = {}
tree = data['tree'][year]

for i, row in data['clusters'].iterrows():
    path = row["path"].split("<br>")
    
    if len(path) >= 2:
        jurisdiction = path[1]
        node = tree.node_at_uuid(row["uuid"])

        try:
            count_by_jurisdiction[path[1]]
        except KeyError:
            count_by_jurisdiction[path[1]] = defaultdict(list)
        
        if row["name"] != jurisdiction:
            if node.range:
                _range = f"{node.unit_class} - {node.range.split()[0]}"
                count_by_jurisdiction[path[1]][_range].append(row["name"])

In [15]:
counts_df = pd.DataFrame({jurisdiction: {_range: int(len(units)) for _range, units in units_by_range.items()} for jurisdiction, units_by_range in count_by_jurisdiction.items()}).T

order = [
    "Sustantiva - Secretaría",
    "Sustantiva - Subsecretaría",
    "Sustantiva - Dirección",
    "Sustantiva - Coordinación",
    "Sustantiva - Escribanía"
]

counts_df = counts_df.reindex(columns=[col for col in order if col in counts_df.columns])
counts_df.astype("Int64")

,Sustantiva - Secretaría,Sustantiva - Subsecretaría,Sustantiva - Dirección,Sustantiva - Coordinación,Sustantiva - Escribanía
Secretaría de Cultura,<NA>,3,31,26,<NA>
Ministerio de Justicia,1,4,46,15,1
Ministerio de Capital Humano,3,9,68,43,<NA>
Ministerio de Economía,13,33,<NA>,<NA>,<NA>
Ministerio de Seguridad Nacional,2,6,44,7,<NA>
"Ministerio de Relaciones Exteriores, Comercio Internacional y Culto",4,4,<NA>,<NA>,<NA>
Ministerio de Defensa,3,4,25,15,<NA>
Ministerio de Salud,2,6,44,11,<NA>
Ministerio de Desregulación y Transformación del Estado,2,5,23,4,<NA>
Jefatura de Gabinete de Ministros,8,16,150,51,<NA>


#### Fragmentación

In [ ]:
# Corroborar equivalencia entre unidades del experimento superposicion y fragmentacion
import json
from chainsaw.db import SessionLocal

with SessionLocal() as session:
    tree = Tree.load_or_create(
            f"data/estructura/{year}",
            session,
            central_administration_only=True,
        )

    with open(f"clusters/clusters_{year.split("_")[0]}_sustantivas_5_v3.json", "r", encoding="utf-8") as f:
        primeros = cluster_file_content = json.load(f)["objectives"].keys()

    segundos = []
    for i, row in data["clusters"].iterrows():
        segundos.append(row["uuid"])

    print("Faltantes en fragmentacion:")
    for uuid in primeros:
        if uuid not in segundos:
            print(tree.node_at_uuid(uuid).name)

    print("-----")

    print("Faltantes en superposicion:")
    for uuid in segundos:
        if uuid not in primeros:
            print(tree.node_at_uuid(uuid).name)

In [ ]:
LEMMAS = {
    "defensa nacional": ["militar", "fuerza armadas", "ejercito", "defensa nacional"],
    "género y diversidad sexual": ["mujer", "feminismo", "sexual", "genero"],
    "hábitat y vivienda": ["vivienda", "habitat", "urbanismo", "urbano", "integracion urbano"],
    "medio ambiente": ["ambiente", "ecologia", "sustentable", "cambio climatico"],
    "política social": ["pobreza", "desarrollo social", "familia"],
    "seguridad interior": ["delito", "seguridad interior", "fuerza seguridad", "seguridad federal", "policia", "narcotrafico"],
    "trabajo y empleo": ["trabajador", "empleo", "laboral", "sindicato", "empleador"]
}

In [ ]:
# Busqueda de lemas para analisis de fragmentacion
import re
import spacy
import numpy as np
import pandas as pd
import unicodedata
from tqdm import tqdm
from collections import defaultdict
from chainsaw.model.official_document import Prompt, Objective
from chainsaw.db import SessionLocal


nlp = spacy.load("es_core_news_sm")


def normalizar_texto(texto: str) -> str:
    texto = texto.lower()
    texto = re.sub(r'\s+', ' ', texto)
    texto = re.sub(r'[^a-z0-9áéíóúüñ ]', '', texto)
    return texto.strip()


def eliminar_verbos(texto: str) -> str:
    doc = nlp(texto)
    tokens = [t.lemma_ for t in doc if t.pos_ != "VERB" and not t.is_stop and t.is_alpha]
    tokens = [''.join(
        c for c in unicodedata.normalize('NFD', tok) if unicodedata.category(c) != 'Mn'
    ) for tok in tokens]
    tokens = [tok for tok in tokens if len(tok) > 3]
    return " ".join(tokens)


def encontrar_subtextos(normalized_text: str, lemmas_dict: dict, window: int = 15):
    words = normalized_text.split()
    results = {}

    for category, lemmas in lemmas_dict.items():
        categorized = False
        for lemma in lemmas:
            if categorized:
                break

            lemma_tokens = lemma.split()
            n = len(lemma_tokens)

            for i in range(len(words) - n + 1):
                if words[i:i+n] == lemma_tokens:
                    start = max(0, i - window)
                    end = min(len(words), i + n + window)
                    context = words[start:end]

                    context_highlighted = context.copy()
                    for j in range(n):
                        context_highlighted[i - start + j] = f"<br><b>{words[i + j]}</b><br>"

                    subtext = " ".join(context_highlighted)
                    results[category] = subtext
                    categorized = True
                    break
    return results


fragmentation = defaultdict(list)


with SessionLocal() as session:
    tree = Tree.load_or_create(
        f"data/estructura/{year}",
        session,
        central_administration_only=True,
    )

    rows = data['clusters'].iterrows()
    for i, row in tqdm(rows, desc="Unidad", leave=False):
        prompt = session.query(Prompt).filter(
            Prompt.tree_id == tree.id,
            Prompt.unit_uuid == row["uuid"],
        ).one_or_none()
        if prompt:
            objective = session.query(Objective).filter(Objective.prompt_id == prompt.id).one_or_none()
            if objective:
                normalized_text = eliminar_verbos(normalizar_texto(f"{row["name"]} {objective.text}"))
                subtxts = encontrar_subtextos(normalized_text, LEMMAS, window=15)

                for category, subtext in subtxts.items():
                    fragmentation[category].append({
                        "uuid": row["uuid"],
                        "subtext": subtext.strip()
                    })
            else:
                raise ValueError(f"No objective found for prompt {prompt.id} of unit {row['uuid']}")
        else:
            raise ValueError(f"No prompt found for unit {row['uuid']}")

filteredData = data.copy()
data['clusters']["category"] = None
data['clusters']["subtext"] = None

# Paso 1: Asociar categorías y subtextos a cada unidad
for category_name, uuid_and_subtexts in fragmentation.items():
    for each in uuid_and_subtexts:
        uuid = each["uuid"]
        subtext = each["subtext"]
        mask = data['clusters']["uuid"] == uuid

        for i in data['clusters'][mask].index:
            if data['clusters'].at[i, "category"] is None:
                data['clusters'].at[i, "category"] = [category_name]
                data['clusters'].at[i, "subtext"] = [subtext]
            else:
                data['clusters'].at[i, "category"].append(category_name)
                data['clusters'].at[i, "subtext"].append(subtext)

# Paso 2: Expandir filas (una por categoría/subtexto)
rows = []
for _, row in data['clusters'].iterrows():
    categories = row["category"]
    subtexts = row["subtext"]

    if categories is not None and subtexts is not None:
        for j, (cat, sub) in enumerate(zip(categories, subtexts)):
            new_row = row.copy()
            new_row["category"] = cat
            new_row["subtext"] = sub
            if "x" in new_row and "y" in new_row:
                new_row["x"] += np.random.uniform(-0.0003, 0.0003)
                new_row["y"] += np.random.uniform(-0.0003, 0.0003)
            rows.append(new_row)

# Paso 3: Crear nuevo DataFrame expandido
filteredData['clusters'] = pd.DataFrame(rows).reset_index(drop=True)

In [ ]:
# Numero efectivo de ministerios por categoria
def min_effective_number(clusters_data):
    categories = set(clusters_data["category"].to_list())

    units_by_category = {category: defaultdict(list) for category in categories}
    for i, row in clusters_data.iterrows():
        category = row["category"]
        if category:
            units_by_category[category][row["jurisdiction"]].append(row["uuid"])
    
    effective_numbers = {}
    for category, units_by_ministry in units_by_category.items():
        total_units = sum(len(units) for units in units_by_ministry.values())
        denominator = 0

        for ministry, units in units_by_ministry.items():
            factor = (len(units) / total_units) ** 2
            denominator = denominator + factor
        effective_numbers[category] = 1 / denominator
    return effective_numbers

min_effective_number(filteredData["clusters"])

In [ ]:
# Guardar lemas en archivo csv
filteredData['clusters'].to_csv(f"fragments{year.split('_')[0]}.csv", index=False)

In [ ]:
# Función de visualización con 'category' y 'subtext'
import plotly.express as px
import plotly.graph_objects as go
import alphashape
from shapely.geometry import Polygon
import pandas as pd

def clusters_scatter_by_category(data, cluster_palette=px.colors.qualitative.Plotly):
    df_clusters = data['clusters']
    title = f"Clusters por categoría ({data['first']['selected_tree']}: {data['first']['units_amount']} uds.)"

    # Paletas de colores
    category_labels = sorted(df_clusters["category"].dropna().unique())
    category_colors = {
        label: cluster_palette[i % len(cluster_palette)]
        for i, label in enumerate(category_labels)
    }

    cluster_ids = [cid for cid in sorted(df_clusters['cluster'].unique()) if cid != -1]
    hull_palette = px.colors.qualitative.Plotly
    cluster_hull_colors = {
        cid: hull_palette[i % len(hull_palette)]
        for i, cid in enumerate(cluster_ids)
    }

    fig = go.Figure()

    # Dibujar las envolventes de cada cluster
    for cluster_id in cluster_ids:
        df_cluster = df_clusters[df_clusters['cluster'] == cluster_id]
        points = df_cluster[['x', 'y']].values

        if len(points) >= 3:
            alpha_shape = alphashape.alphashape(points, 0.1)
            polygons = [alpha_shape] if isinstance(alpha_shape, Polygon) else list(alpha_shape.geoms)

            for poly in polygons:
                x, y = poly.exterior.xy
                fig.add_trace(go.Scatter(
                    x=list(x),
                    y=list(y),
                    mode='lines',
                    line=dict(color=cluster_hull_colors[cluster_id], width=2, dash="dot"),
                    fill='toself',
                    fillcolor=cluster_hull_colors[cluster_id],
                    opacity=0.1,
                    name=f"Cluster {cluster_id}",
                    showlegend=False
                ))

    # Dibujar los puntos por categoría
    for date in df_clusters['date'].unique():
        df_date = df_clusters[df_clusters['date'] == date]

        for category_label in category_labels:
            df_subset = df_date[df_date["category"] == category_label]
            if df_subset.empty:
                continue

            fig.add_trace(go.Scatter(
                x=df_subset['x'],
                y=df_subset['y'],
                mode='markers',
                name=category_label,
                marker=dict(
                    size=10,
                    color=category_colors[category_label],
                    line=dict(width=1),
                    opacity=1,
                ),
                # Mostramos path + subtext en el tooltip
                customdata=df_subset[["path", "subtext"]],
                hovertemplate="<b>%{customdata[0]}</b><br>%{customdata[1]}<extra></extra>",
            ))

    fig.update_layout(
        height=800,
        title=title,
        legend_title_text='Categorías de fragmentación',
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=-0.2,
            xanchor='center',
            x=0.5,
            traceorder='normal'
        ),
    )

    fig.show()


In [ ]:
clusters_scatter_by_category(filteredData, px.colors.qualitative.Light24)

#### Bajar clusters a JSON

In [ ]:
import plotly.express as px

sizes = list(cluster_sizes.values())
fig = px.histogram(sizes, nbins=50,
                   labels={'value':'Tamaño del Cluster'},
                   title=f'Histograma de Tamaños de Clusters ({len(cluster_sizes.keys())} clusters)')

fig.show()

In [ ]:
import plotly.express as px

sizes = list(jurisdictions_by_cluster.values())
fig = px.histogram(sizes, nbins=20,
                   labels={'value':'Cantidad de jurisdicciones'},
                   title=f'Histograma de Variedad de jurisdicciones en los Clusters ({len(jurisdictions_by_cluster.keys())} clusters + Ruido)')

fig.show()

In [ ]:
# Lista de unidades por cluster (para promptear el Label que nombra a cada cluster)
clusters_idxs = data['clusters']["cluster"].unique()
clusters_idxs.sort()
clusters_idxs = clusters_idxs[clusters_idxs != -1]

full_dict = {
    int(idx): [name.split("<br>")[-1] 
               for name in data['clusters'].loc[data['clusters']["cluster"] == idx, "name"].tolist()]
    for idx in clusters_idxs
}

cluster_sizes = {idx: len(full_dict[idx]) for idx in clusters_idxs}
cluster_sizes[-1] = data['clusters'].loc[data['clusters']["cluster"] == -1, "name"].nunique()
jurisdictions_by_cluster = {int(idx):len(set(data['clusters'].loc[data['clusters']["cluster"] == idx, "jurisdiction"].to_list())) for idx in clusters_idxs if idx != -1}

step = 10
chunks = []
for start in range(0, max(clusters_idxs)+1, step):
    end = start + step
    # filtrar las claves que estén en este rango
    chunk_dict = {k: v for k, v in full_dict.items() if start <= k < end}
    if chunk_dict:  # solo agregar si no está vacío
        chunks.append(chunk_dict)

# PROMPT para labels:
prompt = "Dado el siguiente JSON cuyas claves son un indice y los valores una lista de unidades estatales, para cada lista generá un string de no más de 3 palabras, que sea representativo del tópico de dichas unidades. Finalmente, devolvé un JSON sin saltos de línea, donde la clave sea el índice original de la lista, y el valor, el string para el tópico identificado. No agregues enunciado, explicación o texto alguno, solo el JSON:"

for i, c in enumerate(chunks):
    print(f"{prompt} {c}")

In [ ]:
# Las líneas output de arriba son prompts que se deben ejecutar en ChatGPT Web (o tu LLM fav) para obtener los nombres de los clusters.
# Los JSON resultantes se deben mergear en clusters_ids
a = {"0":"hogares asistenciales","1":"protección social","2":"políticas familiares","3":"educación pública","4":"trabajo y empleo"}
clusters_ids = a

In [ ]:
# Descargar archivo JSON de clusters para visualizar en el viewer online
import os
import json
from chainsaw.db import SessionLocal
from chainsaw.model.official_document import Prompt, Objective

output_file_name = f"clusters_{year}_sustantivas_5.json"

objectives = {}
with SessionLocal() as session:
    tree = session.query(Tree).filter(
        Tree.date_string.startswith(year),
        Tree.central_administration_only == True,
    ).one()

    uuids = data['clusters']['uuid'].tolist()
    results = (
        session.query(Objective, Prompt.unit_uuid)
        .join(Prompt, Objective.prompt_id == Prompt.id)
        .filter(
            Prompt.unit_uuid.in_(uuids),
            Prompt.tree_id == tree.id
        )
        .all()
    )

    objectives = {unit_uuid: obj.text for obj, unit_uuid in results}

data_to_export = {
    "clusters": clusters_ids,
    "clusters_data": data['clusters'].to_dict(orient="records"),
    "objectives": objectives
}

with open(os.path.join(output_file_name), "w", encoding="utf-8") as f:
    json.dump(data_to_export, f, ensure_ascii=False, indent=2)

#### Obtener responsabilidades primarias

In [ ]:
# Status actual
import pandas as pd
import plotly.express as px
from collections import defaultdict
from chainsaw.db import SessionLocal
from chainsaw.model.official_document import Objective, Prompt

rows = []
with SessionLocal() as session:
    for date, tree in data['tree'].items():
        units_count = len(tree.units)

        prompts_count = session.query(Prompt).filter(Prompt.tree_id == tree.id).count()

        objectives_count = (
            session.query(Objective)
            .join(Prompt, Prompt.id == Objective.prompt_id)
            .filter(Prompt.tree_id == tree.id)
            .count()
        )

        restantes_count = units_count - objectives_count

        rows.extend([
            {"Fecha": date, "Tipo": "Unidades", "Cantidad": units_count},
            {"Fecha": date, "Tipo": "Prompts", "Cantidad": prompts_count},
            {"Fecha": date, "Tipo": "Objetivos", "Cantidad": objectives_count},
            {"Fecha": date, "Tipo": "Restantes", "Cantidad": restantes_count}
        ])

df = pd.DataFrame(rows)

fig = px.bar(
    df,
    x="Fecha",
    y="Cantidad",
    color="Tipo",
    barmode="group",
    title="Unidades, Prompts, Objetivos y Restantes por Fecha",
    text="Cantidad",
)

fig.update_traces(textposition="outside")
fig.update_layout(width=1000, height=400, yaxis=dict(range=[0, 3500]))
fig.show()

In [ ]:
# Restantes
import pandas as pd
import plotly.graph_objects as go
from IPython.display import display
from collections import Counter
from sqlalchemy import exists, select
from ipywidgets import HTML, Layout
from chainsaw.db import SessionLocal
from chainsaw.model.official_document import Objective, Prompt, Unit

rows = []
names_dict = {}

with SessionLocal() as session:
    for date, tree in data['tree'].items():
        subq = (
            select(Objective.id)
            .join(Prompt, Prompt.id == Objective.prompt_id)
            .where(
                Prompt.unit_uuid == Unit.uuid,
                Prompt.tree_id == tree.id,
            )
            .correlate(Unit)
        )

        units_without_objectives = (
            session.query(Unit)
            .filter(
                Unit.tree_id == tree.id,
                ~exists(subq),
            )
            .all()
        )

        types = [unit.name.split()[0] for unit in units_without_objectives]
        counter = Counter(types)

        for name, count in counter.most_common():
            rows.append({"Fecha": date, "Tipo": name, "Cantidad": count})
            full_names = sorted(
                [f"{unit.name} | {unit.uuid}" 
                for unit in units_without_objectives 
                if unit.name.split()[0] == name]
            )
            names_dict[(date, name)] = "<br>".join(full_names)

df = pd.DataFrame(rows)
first_date = df["Fecha"].sort_values().iloc[0]
first_df = df[df["Fecha"] == first_date]
tipo_order = first_df.sort_values("Cantidad", ascending=False)["Tipo"].tolist()

fig = go.FigureWidget()
for tipo in tipo_order:
    sub_df = df[df["Tipo"] == tipo]
    fig.add_bar(
        x=sub_df["Fecha"],
        y=sub_df["Cantidad"],
        name=tipo,
        customdata=[(r["Fecha"], r["Tipo"]) for _, r in sub_df.iterrows()],
    )

fig.update_layout(
    barmode="group",
    title="Restantes por Tipo y Fecha",
    width=1000,
    height=450
)

output = HTML(
    value="Hacé click en una barra para ver los nombres aquí.",
    layout=Layout(
        height="200px",
        overflow_y="scroll",
        width="100%",
    )
)

def on_click(trace, points, selector):
    if points.point_inds:
        ind = points.point_inds[0]
        date, tipo = trace.customdata[ind]
        output.value = names_dict[(date, tipo)]

for trace in fig.data:
    trace.on_click(on_click)

display(fig, output)

In [ ]:
# descargar csv de unidades sin prompts/sin funciones
import csv
from sqlalchemy import and_
from chainsaw.db import SessionLocal
from chainsaw.model.official_document import Objective, Prompt
from chainsaw.enum.administration_type import AdministrationType


with SessionLocal() as session:
    for date, tree in data['tree'].items():
        units_without_prompt = (
            session.query(Unit)
            .outerjoin(Prompt, and_(
                Prompt.tree_id == Unit.tree_id,
                Prompt.unit_uuid == Unit.uuid
            ))
            .filter(
                Unit.tree_id == tree.id,
                Unit.type == AdministrationType.CENTRAL_ADMINISTRATION.value,
                Prompt.id.is_(None)
            )
            .all()
        )

        units_without_objectives_but_prompt = (
            session.query(Unit)
            .outerjoin(Prompt, and_(
                Prompt.tree_id == Unit.tree_id,
                Prompt.unit_uuid == Unit.uuid
            ))
            .outerjoin(Objective, and_(
                Objective.prompt_id == Prompt.id,
            ))
            .filter(
                Unit.tree_id == tree.id,
                Unit.type == AdministrationType.CENTRAL_ADMINISTRATION.value,
                Objective.id.is_(None),
                Prompt.id.isnot(None),
            )
            .all()
        )

        with open(f"unidades_sin_prompt_{date}.csv", "w", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow(["id", "name", "path"])
            for unit in units_without_prompt:
                path = tree.path_format(tree.path_to(unit.uuid))
                writer.writerow([unit.uuid, unit.name, path])
        
        with open(f"unidades_sin_funciones_{date}.csv", "w", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow(["id", "name", "path"])
            for unit in units_without_objectives_but_prompt:
                path = tree.path_format(tree.path_to(unit.uuid))
                writer.writerow([unit.uuid, unit.name, path])

In [ ]:
# Ejecutar procesamiento de documentos oficiales

for date in ("2023_12_09", "2025_07_08"):
    with SessionLocal() as session:
        Pipeline.start(
            session,
            tree=data['tree'][date],
            steps=[
                # Scrapping(),
                # Cleaning(),
                # Finding(),
                # Prompting(),
                # LLMExtraction(processes_amount=5),
            ],
            override=False,
        )